In [132]:
import joblib
import time

from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV


from sklearn.svm import SVC
import xlrd
import pandas as pd
import numpy as np
import openpyxl
from nltk import word_tokenize

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report, cohen_kappa_score, f1_score
import torch
import tensorflow as tf
from transformers import AutoTokenizer, AutoModel, TrainingArguments, Trainer, AutoModelForSequenceClassification, EarlyStoppingCallback


from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB



from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import cross_validate, GridSearchCV
from sklearn import metrics

from sklearn import metrics


## Import Datasets

In [34]:
# The original question dataset generated by GPT without LWIC 
secondary = pd.read_excel("../data/secondaryquestionsonly.xlsx")
pka = pd.read_excel("../data/pKa_blooms.xlsx")

In [35]:
secondary=secondary.drop(columns=['Unnamed: 0'])
pka=pka.drop(columns=['Unnamed: 0'])
pka = pka.drop_duplicates(subset='question_ID', keep='first')

result = pd.concat([secondary, pka], ignore_index=True)

In [36]:
result.to_excel('all_generated_questions.xlsx')

In [37]:
result.head()

,question,label,question_ID
0,In this reaction which compound acts as the r...,2,34
1,Consider a scenario where Compound C is oxidiz...,1,32
2,In the context of biological redox reactions ...,0,27
3,Which assumption is necessary for biologists' ...,3,36
4,Which of the following best describes the role...,2,35


In [ ]:
# Import LWIC for generated questions and for 

In [43]:
# Importing the LWIC features for our generated questions
lwic_combined = pd.read_csv("LIWC_all_generated_questions .csv")
lwic_combined
lwic_combined = lwic_combined.drop(columns=["Segment","Unnamed: 0"])


In [45]:
# Make the whole question lower case
lwic_combined['question'] = lwic_combined['question'].str.lower()

In [46]:
lwic_combined.head()

,question,label,question_ID,WC,Analytic,Clout,Authentic,Tone,WPS,BigWords,...,nonflu,filler,AllPunc,Period,Comma,QMark,Exclam,Apostro,OtherP,Emoji
0,in this reaction which compound acts as the r...,2,34,9,89.52,2.75,77.17,NaN,9.0,33.33,...,0,0,11.11,0.00,0.0,11.11,0,0.00,0.0,0
1,consider a scenario where compound c is oxidiz...,1,32,35,72.76,40.06,42.95,65.27,17.5,34.29,...,0,0,5.71,2.86,0.0,2.86,0,0.00,0.0,0
2,in the context of biological redox reactions ...,0,27,14,99.00,NaN,24.32,NaN,14.0,28.57,...,0,0,21.43,0.00,0.0,7.14,0,14.29,0.0,0
3,which assumption is necessary for biologists' ...,3,36,19,98.00,40.06,70.28,92.27,19.0,47.37,...,0,0,10.53,0.00,0.0,5.26,0,5.26,0.0,0
4,which of the following best describes the role...,2,35,14,99.00,40.06,24.32,NaN,14.0,28.57,...,0,0,7.14,0.00,0.0,7.14,0,0.00,0.0,0


In [53]:
lwic_combined= lwic_combined.fillna(0.0)


## Read Data For Training


In [47]:
training_data = pd.read_csv('LIWC_combined_questions.csv', delimiter=',', skipinitialspace=True, quotechar='"')

In [50]:
training_data['question'] =training_data['question'].str.lower()

In [51]:
training_data.columns

Index(['question', 'label', 'Segment', 'WC', 'Analytic', 'Clout', 'Authentic',
       'Tone', 'WPS', 'BigWords',
       ...
       'nonflu', 'filler', 'AllPunc', 'Period', 'Comma', 'QMark', 'Exclam',
       'Apostro', 'OtherP', 'Emoji'],
      dtype='object', length=121)

In [52]:
training_data = training_data.drop(columns=["Segment"])

In [556]:
data

,Learning_outcome,bloom_one_hot,bloom
0,Analyze the health economic implications of ef...,"[0, 0, 0, 1]",3
1,Apply research skills to operate effectively a...,"[0, 0, 1, 0]",2
3,Describe the general characteristics of the m...,"[0, 1, 0, 0]",1
5,explain key terms and concepts used to engage ...,"[0, 1, 0, 0]",1
6,Identify an issue of relevance to the practice...,"[0, 0, 0, 1]",3
...,...,...,...
21296,write a coherent piece of advice discussing th...,"[0, 1, 0, 0]",1
21321,Write an essay using a range of Japanese sourc...,"[0, 0, 1, 0]",2
21374,"Write/type simple sentences using hiragana, ka...","[0, 0, 1, 0]",2
21375,"Write/type simple sentences using hiragana, ka...","[0, 0, 1, 0]",2


In [55]:
training_data= training_data.fillna(0.0)

In [565]:
data['Learning_outcome']=data['Learning_outcome'].str.lower()

### |Multiclass Models

In [56]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB

In [57]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier

In [58]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import cross_validate, GridSearchCV
from sklearn import metrics

In [59]:
import matplotlib.pyplot as plt

In [60]:
def generateX(data_x, test_x, textual_column_index, start_index_LIWC, end_index_LIWC):
    # generating ML features based on previous literature
    # Can try to run using less features for storage
    column_names = []
    print("Getting Unigram...")
    uni_cv = CountVectorizer(stop_words='english', ngram_range=(1, 1), max_features=1000)
    unigram = uni_cv.fit_transform(data_x[:, textual_column_index])
    unigram = unigram.toarray()
    unigram_test = uni_cv.transform(test_x[:,textual_column_index]).toarray()
    temp = uni_cv.get_feature_names_out().tolist()
    column_names += ["uni_"+name for name in temp]
    print("Getting Bigram...")
    bi_cv = CountVectorizer(stop_words='english', ngram_range=(2, 2), max_features=1000)
    bigram = bi_cv.fit_transform(data_x[:, textual_column_index])
    bigram = bigram.toarray()
    bigram_test = bi_cv.transform(test_x[:, textual_column_index]).toarray()
    temp = bi_cv.get_feature_names_out().tolist()
    column_names += ["bi_"+name for name in temp]
    print("Getting Tfidf...")
    tfidf = TfidfVectorizer(stop_words='english', ngram_range=(1, 1), max_features=1000)
    t = tfidf.fit_transform(data_x[:, textual_column_index])
    t = t.toarray()
    t_test = tfidf.transform(test_x[:, textual_column_index]).toarray()
    temp = tfidf.get_feature_names_out().tolist()
    column_names += ["tfidf_"+name for name in temp]
    print("Getting ARI...")
    ari = [textstat.automated_readability_index(text) for text in data_x[:, textual_column_index]]
    ari_test = [textstat.automated_readability_index(text) for text in test_x[:, textual_column_index]]
    column_names.append("ari")
    combined_data_x = []
    combined_test_x = []
    print("Combining...")
    for i in range(len(data_x)):
        combined_data_x.append(unigram[i].tolist()
                              + bigram[i].tolist()
                              + t[i].tolist()
                              + [ari[i]]
                              + data_x[i, start_index_LIWC:end_index_LIWC].tolist())
    for i in range(len(test_x)):
        combined_test_x.append(unigram_test[i].tolist()
                              + bigram_test[i].tolist()
                              + t_test[i].tolist()
                              + [ari_test[i]]
                              + test_x[i, start_index_LIWC:end_index_LIWC].tolist())
    print("Generated feature shape is", np.array(combined_data_x).shape)
    print("Generated test feature is", np.array(combined_test_x).shape)
    return combined_data_x, column_names, combined_test_x, uni_cv, bi_cv, tfidf

In [61]:
## This is necessary to ensure that we can convert our questions into something that can be compare
def transformX(test_x,textual_column_index,start_index_LIWC, end_index_LIWC,uni_cv, bi_cv, tfidf):
    column_names = []
    print("Getting Unigram...")
    unigram_test = uni_cv.transform(test_x[:,textual_column_index]).toarray()
    temp = uni_cv.get_feature_names_out().tolist()
    column_names += ["uni_"+name for name in temp]
    print("Getting Bigram...")
    bigram_test = bi_cv.transform(test_x[:, textual_column_index]).toarray()
    temp = bi_cv.get_feature_names_out().tolist()
    column_names += ["bi_"+name for name in temp]
    print("Getting Tfidf...")
    t_test = tfidf.transform(test_x[:, textual_column_index]).toarray()
    temp = tfidf.get_feature_names_out().tolist()
    column_names += ["tfidf_"+name for name in temp]
    print("Getting ARI...")
    
    ari_test = [textstat.automated_readability_index(text) for text in test_x[:, textual_column_index]]
    column_names.append("ari")
    
    combined_test_x = []
    print("Combining...")
 
    for i in range(len(test_x)):
        combined_test_x.append(unigram_test[i].tolist()
                              + bigram_test[i].tolist()
                              + t_test[i].tolist()
                              + [ari_test[i]]
                              + test_x[i, start_index_LIWC:end_index_LIWC].tolist())
    print("Generated test feature is", np.array(combined_test_x).shape)

    return combined_test_x
    
    
    

In [62]:
def performancePrinter(test_y, pred_y, y_prob=None):
    # Print accuracy score
    print("Accuracy Score -> ", accuracy_score(test_y, pred_y))
    
    # Print Cohen's Kappa Score
    print("Kappa Score -> ", cohen_kappa_score(test_y, pred_y))
    
    # Check if ROC AUC can be calculated (requires probability scores)
    if y_prob is not None:
        try:
            # Calculate ROC AUC score for multi-class classification
            print("ROC AUC Score -> ", roc_auc_score(test_y, y_prob, multi_class='ovr', average='macro'))
        except ValueError as e:
            print(f"Error calculating ROC AUC Score: {e}")
    else:
        print("ROC AUC Score -> Cannot calculate (probability scores required)")
    
    # Print F1 Score with macro averaging for multi-class classification
    print("F1 Score -> ", f1_score(test_y, pred_y, average='macro'))
    
    # Print classification report
    print("Classification report -> \n", classification_report(test_y, pred_y))

### Grid Search Parameters


In [63]:
params_nb = {'var_smoothing': [1e-8, 1e-9, 1e-10]}
params_svm = {'C': [0.1, 1, 10, 100],
              'gamma': ['scale', 'auto'],
              'kernel': ['linear', 'poly', 'rbf']}
params_lr = {'penalty': ['l1', 'l2', 'none'],
             'C': [0.1, 1, 10],
             'solver': ['saga','liblinear'],
             'tol': [0.01, 0.001, 0.0001],
             'max_iter': [200, 500]}

params_rf = {'n_estimators': [50, 100, 250],
             'max_depth': [None, 5, 10],
             'max_features':['auto', 'sqrt'],
             'min_samples_split': [2, 5, 10],
             'min_samples_leaf': [1, 2, 4],
             'bootstrap': [True, False]}

params_xgb = {'gamma':[0.1, 0.5],
              'learning_rate': [0.1, 0.5],
              'max_depth': [5, 7, 10],
              'n_estimators': [50, 100]}

In [65]:
training_data.columns

Index(['question', 'label', 'WC', 'Analytic', 'Clout', 'Authentic', 'Tone',
       'WPS', 'BigWords', 'Dic',
       ...
       'nonflu', 'filler', 'AllPunc', 'Period', 'Comma', 'QMark', 'Exclam',
       'Apostro', 'OtherP', 'Emoji'],
      dtype='object', length=120)

In [67]:
training_data['label'].unique()

array([0, 1, 2, 3])

In [68]:
training_data

,question,label,WC,Analytic,Clout,Authentic,Tone,WPS,BigWords,Dic,...,nonflu,filler,AllPunc,Period,Comma,QMark,Exclam,Apostro,OtherP,Emoji
0,can you list the ingredients needed for a clas...,0,11,97.37,99.00,1.00,0.0,11.0,27.27,72.73,...,0.0,0,9.09,0.00,0.00,9.09,0,0.0,0.0,0
1,what are the top five tourist attractions in p...,0,9,89.52,0.00,77.17,0.0,9.0,22.22,66.67,...,0.0,0,11.11,0.00,0.00,11.11,0,0.0,0.0,0
2,list five common programming languages.,0,5,0.00,0.00,0.00,0.0,5.0,40.00,80.00,...,0.0,0,20.00,20.00,0.00,0.00,0,0.0,0.0,0
3,name three types of renewable energy sources.,0,7,99.00,97.11,0.00,0.0,7.0,28.57,57.14,...,0.0,0,14.29,14.29,0.00,0.00,0,0.0,0.0,0
4,can you list the planets in our solar system?,0,9,66.04,99.00,1.00,0.0,9.0,11.11,55.56,...,0.0,0,11.11,0.00,0.00,11.11,0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2431,what are thrombocytes more accurately called?,0,6,10.19,0.00,0.00,0.0,6.0,33.33,66.67,...,0.0,0,16.67,0.00,0.00,16.67,0,0.0,0.0,0
2432,what is the first step in hemostasis?,0,7,89.52,0.00,97.09,0.0,7.0,14.29,85.71,...,0.0,0,14.29,0.00,0.00,14.29,0,0.0,0.0,0
2433,during which is prothrombin converted to throm...,0,7,89.52,1.00,97.09,0.0,7.0,42.86,57.14,...,0.0,0,14.29,0.00,0.00,14.29,0,0.0,0.0,0
2434,what is the process called in which antibodies...,0,19,99.00,14.81,8.42,0.0,19.0,26.32,57.89,...,0.0,0,10.53,0.00,5.26,5.26,0,0.0,0.0,0


In [69]:
# Split data into training and test sets
split_train_x, split_test_x, split_train_y, split_test_y = train_test_split(training_data.drop(columns=list(training_data.columns[1:2])), training_data[training_data.columns[1:2]], test_size=0.2, random_state=666)


In [70]:
split_train_x.columns

Index(['question', 'WC', 'Analytic', 'Clout', 'Authentic', 'Tone', 'WPS',
       'BigWords', 'Dic', 'Linguistic',
       ...
       'nonflu', 'filler', 'AllPunc', 'Period', 'Comma', 'QMark', 'Exclam',
       'Apostro', 'OtherP', 'Emoji'],
      dtype='object', length=119)

In [71]:
lwic_combined


,question,label,question_ID,WC,Analytic,Clout,Authentic,Tone,WPS,BigWords,...,nonflu,filler,AllPunc,Period,Comma,QMark,Exclam,Apostro,OtherP,Emoji
0,in this reaction which compound acts as the r...,2,34,9,89.52,2.75,77.17,0.00,9.00,33.33,...,0,0,11.11,0.00,0.0,11.11,0,0.00,0.00,0
1,consider a scenario where compound c is oxidiz...,1,32,35,72.76,40.06,42.95,65.27,17.50,34.29,...,0,0,5.71,2.86,0.0,2.86,0,0.00,0.00,0
2,in the context of biological redox reactions ...,0,27,14,99.00,0.00,24.32,0.00,14.00,28.57,...,0,0,21.43,0.00,0.0,7.14,0,14.29,0.00,0
3,which assumption is necessary for biologists' ...,3,36,19,98.00,40.06,70.28,92.27,19.00,47.37,...,0,0,10.53,0.00,0.0,5.26,0,5.26,0.00,0
4,which of the following best describes the role...,2,35,14,99.00,40.06,24.32,0.00,14.00,28.57,...,0,0,7.14,0.00,0.0,7.14,0,0.00,0.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,a molecule has a functional group with a pka o...,3,23,27,99.00,80.61,0.00,0.00,13.50,18.52,...,0,0,22.22,22.22,0.0,0.00,0,0.00,0.00,0
76,which statement best predicts the behavior of ...,0,24,19,99.00,40.06,8.42,0.00,19.00,26.32,...,0,0,21.05,0.00,0.0,5.26,0,0.00,15.79,0
77,a scientist is studying a biomolecule with a c...,1,25,41,99.00,88.82,1.00,0.00,13.67,31.71,...,0,0,14.63,4.88,0.0,2.44,0,0.00,7.32,0
78,if the enzyme is placed in a solution with a p...,2,26,31,95.94,40.06,35.88,0.00,31.00,16.13,...,0,0,9.68,3.23,0.0,3.23,0,3.23,0.00,0


In [80]:
# Adjusting our data to match
lwic_x = lwic_combined.drop(columns=list(lwic_combined.columns[1:2]))
lwic_y = lwic_combined[lwic_combined.columns[1:2]]

In [81]:
lwic_x.shape, lwic_y.shape, split_train_y.shape, split_train_x.shape

((80, 120), (80, 1), (1948, 1), (1948, 119))

In [84]:
split_train_x.shape

(1948, 119)

In [85]:
## Prepare Data for models

x_train, y_train = split_train_x.to_numpy(), split_train_y['label'].astype('long').to_numpy()
x_auto, y_auto  = lwic_x.to_numpy(), lwic_y['label'].astype('long').to_numpy()

In [86]:
np.unique(y_train)

array([0, 1, 2, 3])

In [87]:
split_test_y

,label
579,2
1881,0
46,0
752,2
1489,0
...,...
1578,0
534,1
582,2
2198,0


In [88]:
#Traditional ML Algorithms

In [89]:

#pip install textstat

In [90]:
import textstat

In [92]:
combined_x, column_names, test_x, uni_cv, bi_cv,tfifd = generateX(x_train, split_test_x.to_numpy(), 0, 1, 119)
train_x = combined_x
train_y = y_train
#modifying here to use one hot
test_y = split_test_y['label'].astype('long').to_numpy()

Getting Unigram...
Getting Bigram...
Getting Tfidf...
Getting ARI...
Combining...
Generated feature shape is (1948, 3119)
Generated test feature is (488, 3119)


In [93]:
combined_x_auto = transformX(x_auto,0,1,119, uni_cv, bi_cv, tfifd)

Getting Unigram...
Getting Bigram...
Getting Tfidf...
Getting ARI...
Combining...
Generated test feature is (80, 3119)


In [94]:
len(combined_x[0])

3119

In [95]:
y_auto

array([2, 1, 0, 3, 2, 2, 1, 3, 0, 1, 0, 3, 0, 1, 2, 3, 1, 0, 3, 3, 1, 0,
       2, 2, 1, 2, 0, 3, 0, 1, 2, 3, 2, 0, 0, 2, 3, 1, 1, 3, 3, 1, 0, 2,
       0, 1, 2, 3, 1, 0, 1, 2, 1, 3, 3, 2, 3, 0, 2, 0, 0, 1, 2, 3, 1, 2,
       3, 0, 0, 1, 2, 3, 1, 0, 2, 3, 0, 1, 2, 3])

In [96]:
scaler = StandardScaler()
train_x_scaled = scaler.fit_transform(train_x)
test_x_scaled = scaler.transform(test_x)
auto_x_scaled = scaler.transform(combined_x_auto)

NameError: name 'StandardScaler' is not defined

In [601]:
len(train_y)

10680

In [602]:
len(test_y)

2670

In [523]:
len(y_auto)

50

In [608]:
len(train_x)

10680

In [609]:
len(train_y)

10680

In [610]:
len(test_x)

2670

In [611]:
len(test_y)


2670

In [612]:
len(combined_x_auto)

50

In [97]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
train_x_scaled = scaler.fit_transform(train_x)
test_x_scaled = scaler.transform(test_x)
auto_x_scaled = scaler.transform(combined_x_auto)

In [98]:
import os 
import pickle


In [100]:
features = [train_x, train_y, test_x, test_y, combined_x_auto, y_auto, train_x_scaled, test_x_scaled, auto_x_scaled]
feature_names = ['train_x', 'train_y', 'test_x', 'test_y', 'x_auto', 'y_auto', 'train_x_scaled', 'test_x_scaled', 'auto_x_scaled']
feature_folder = 'features'
for feature, name in zip(features, feature_names):
    pickle_file = os.path.join(feature_folder, name +'.pkl')

    with open(pickle_file, 'wb') as file:
        pickle.dump(feature, file)

    print(f'{name} data has been saved to {pickle_file}')
    

train_x data has been saved to features/train_x.pkl
train_y data has been saved to features/train_y.pkl
test_x data has been saved to features/test_x.pkl
test_y data has been saved to features/test_y.pkl
x_auto data has been saved to features/x_auto.pkl
y_auto data has been saved to features/y_auto.pkl
train_x_scaled data has been saved to features/train_x_scaled.pkl
test_x_scaled data has been saved to features/test_x_scaled.pkl
auto_x_scaled data has been saved to features/auto_x_scaled.pkl


In [101]:
features_folder = 'features'

pickle_file = os.path.join(features_folder, 'train_x.pkl')

## Naive Bayes

In [102]:
import time

from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

from sklearn.svm import SVC

In [103]:
from sklearn.metrics import accuracy_score

In [104]:
gnb = GaussianNB()
gnb_gs = GridSearchCV(gnb, params_nb, scoring="f1_macro", n_jobs=-1, cv=3, verbose=3)
start_time = time.time()
gnb_gs.fit(train_x, train_y)
end_time = time.time()
print(f"Grid Search took {end_time - start_time:.2f} seconds")
pred_y_gnb = gnb_gs.predict(test_x)
print(f"Best parameters: {gnb_gs.best_params_}")
print(f"Best score: {gnb_gs.best_score_}")

accuracy = accuracy_score(test_y, pred_y_gnb)
print(f"Accuracy on the test set: {accuracy:.3f}")

Fitting 3 folds for each of 3 candidates, totalling 9 fits
Grid Search took 9.46 seconds
Best parameters: {'var_smoothing': 1e-08}
Best score: 0.6186947420984906
Accuracy on the test set: 0.592


In [105]:
auto_predictions = gnb_gs.predict(combined_x_auto)
accuracy = accuracy_score(y_auto, auto_predictions)
print(f"Accuracy on Generated Questions: {accuracy:.2f}")

Accuracy on Generated Questions: 0.24


In [252]:
# Will test again with dimensionality reduction

In [106]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [ ]:
params_nb = {'var_smoothing': [1e-8, 1e-9, 1e-10]}
params_svm = {'C': [0.1, 1, 10, 100],
              'gamma': ['scale', 'auto'],
              'kernel': ['linear', 'poly', 'rbf']}
params_lr = {'penalty': ['l1', 'l2', 'none'],
             'C': [0.1, 1, 10],
             'solver': ['saga','liblinear'],
             'tol': [0.01, 0.001, 0.0001],
             'max_iter': [200, 500]}

params_rf = {'n_estimators': [50, 100, 250],
             'max_depth': [None, 5, 10],
             'max_features':['auto', 'sqrt'],
             'min_samples_split': [2, 5, 10],
             'min_samples_leaf': [1, 2, 4],
             'bootstrap': [True, False]}

params_xgb = {'gamma':[0.1, 0.5],
              'learning_rate': [0.1, 0.5],
              'max_depth': [5, 7, 10],
              'n_estimators': [50, 100]}

In [107]:
# First standardize the data
scaler = StandardScaler()
train_x_scaled = scaler.fit_transform(train_x)
test_x_scaled = scaler.transform(test_x)


In [108]:
# Initialize PCA - Choose the number of components (e.g., 50 components)
pca = PCA(n_components=1500)  # Adjust number of components as needed
train_x_pca = pca.fit_transform(train_x_scaled)
test_x_pca = pca.transform(test_x_scaled)


In [109]:
print(f"Original training data shape: {len(train_x[0])}")
print(f"Reduced training data shape: {len(train_x_pca[0])}")
print(f"Original test data shape: {len(test_x[0])}")
print(f"Reduced test data shape: {len(test_x_pca[0])}")


Original training data shape: 3119
Reduced training data shape: 1500
Original test data shape: 3119
Reduced test data shape: 1500


In [165]:
gnb = GaussianNB()
gnb_gs = GridSearchCV(gnb, params_nb, scoring="f1_macro", n_jobs=-1, cv=3, verbose=3)
start_time = time.time()
gnb_gs.fit(train_x_pca, train_y)
end_time = time.time()
print(f"Grid Search took {end_time - start_time:.2f} seconds")
pred_y_gnb = gnb_gs.predict(test_x_pca)
print(f"Best parameters: {gnb_gs.best_params_}")
print(f"Best score: {gnb_gs.best_score_}")

accuracy = accuracy_score(test_y, pred_y_gnb)
print(f"Accuracy on the test set: {accuracy:.2f}")

Fitting 3 folds for each of 3 candidates, totalling 9 fits
Grid Search took 0.45 seconds
Best parameters: {'var_smoothing': 1e-08}
Best score: 0.46163090994178874
Accuracy on the test set: 0.41


In [167]:
performancePrinter(y_auto, pred_y_gnb)

Accuracy Score ->  0.25
Kappa Score ->  0.0
ROC AUC Score -> Cannot calculate (probability scores required)
F1 Score ->  0.1
Classification report -> 
               precision    recall  f1-score   support

           0       0.00      0.00      0.00        20
           1       0.25      1.00      0.40        20
           2       0.00      0.00      0.00        20
           3       0.00      0.00      0.00        20

    accuracy                           0.25        80
   macro avg       0.06      0.25      0.10        80
weighted avg       0.06      0.25      0.10        80



/opt/anaconda3/envs/Diss/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/envs/Diss/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/envs/Diss/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [171]:
len(predictions)

80

In [172]:
result['gaussian_pred'] = predictions

In [173]:
result

,question,label,question_ID,gaussian_pred,SVM_pred,lr_pred,rf_pred,xgb_pred
0,In this reaction which compound acts as the r...,2,34,0,0,1,0,0
1,Consider a scenario where Compound C is oxidiz...,1,32,0,0,2,0,1
2,In the context of biological redox reactions ...,0,27,0,0,1,0,2
3,Which assumption is necessary for biologists' ...,3,36,1,0,2,0,1
4,Which of the following best describes the role...,2,35,0,0,1,0,0
...,...,...,...,...,...,...,...,...
75,A molecule has a functional group with a pKa o...,3,23,0,2,1,0,1
76,Which statement best predicts the behavior of ...,0,24,0,0,1,0,1
77,A scientist is studying a biomolecule with a c...,1,25,0,2,1,0,1
78,If the enzyme is placed in a solution with a p...,2,26,0,2,2,0,2


## Support Vector Machine

In [174]:
import time
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
svc = SVC()

# Setup Grid Search with cross-validation
grid_search = GridSearchCV(svc, params_svm, scoring="f1_macro", n_jobs=-1, cv=3, verbose=3)

# Measure the time taken to fit the model
start_time = time.time()
grid_search.fit(train_x, train_y)
end_time = time.time()

# Print the time taken
print(f"Grid Search took {end_time - start_time:.2f} seconds")

# Predict with the optimized model
pred_y_svc = grid_search.predict(test_x)

# Output the best parameters and score from the GridSearchCV
print(f"Best parameters: {grid_search.best_params_}")
print(f"Best score: {grid_search.best_score_:.2f}")

# Calculate and print the accuracy on the test set
accuracy = accuracy_score(test_y, pred_y_svc)
print(f"Accuracy on the test set: {accuracy:.2f}")

Fitting 3 folds for each of 24 candidates, totalling 72 fits
[CV 3/3] END bootstrap=False, max_depth=5, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=250;, score=0.551 total time=   1.9s
[CV 2/3] END bootstrap=False, max_depth=5, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=50;, score=0.528 total time=   0.7s
[CV 1/3] END bootstrap=False, max_depth=5, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=250;, score=0.570 total time=   1.5s
[CV 3/3] END bootstrap=False, max_depth=5, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=250;, score=0.510 total time=   1.6s
[CV 1/3] END bootstrap=False, max_depth=5, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=50;, score=0.548 total time=   0.6s
[CV 3/3] END bootstrap=False, max_depth=5, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=100;, score=0.531 total time=   0.9s
[CV 3/3] END bootst

In [139]:
pred_y_svc = grid_search.predict(combined_x_auto)
accuracy = accuracy_score(y_auto, pred_y_svc)
print(f"Accuracy on Generated test set: {accuracy:.2f} \n \n")

Accuracy on Generated test set: 0.29 
 

[CV 3/3] END ...C=0.1, gamma=scale, kernel=poly;, score=0.319 total time=  17.0s
[CV 2/3] END ..C=0.1, gamma=auto, kernel=linear;, score=0.722 total time=  20.9s
[CV 1/3] END .....C=1, gamma=scale, kernel=poly;, score=0.632 total time=  14.6s
[CV 1/3] END ......C=1, gamma=scale, kernel=rbf;, score=0.568 total time=  14.5s
[CV 3/3] END ....C=1, gamma=auto, kernel=linear;, score=0.745 total time=  35.8s
[CV 2/3] END .....C=10, gamma=scale, kernel=rbf;, score=0.679 total time=  13.5s
[CV 1/3] END .....C=10, gamma=auto, kernel=poly;, score=0.630 total time=   9.3s
[CV 1/3] END ......C=10, gamma=auto, kernel=rbf;, score=0.654 total time=  20.0s
[CV 1/3] END ...C=100, gamma=scale, kernel=poly;, score=0.662 total time=  10.9s
[CV 3/3] END ....C=100, gamma=scale, kernel=rbf;, score=0.696 total time=  12.6s
[CV 3/3] END ....C=100, gamma=auto, kernel=poly;, score=0.670 total time=  11.5s
[CV 1/3] END ...C=0.1, gamma=scale, kernel=poly;, score=0.314 total 

In [140]:
result['SVM_pred']= pred_y_svc

[CV 1/3] END .C=0.1, gamma=scale, kernel=linear;, score=0.718 total time=  20.3s
[CV 3/3] END ..C=0.1, gamma=auto, kernel=linear;, score=0.733 total time=  23.3s
[CV 2/3] END .....C=1, gamma=scale, kernel=poly;, score=0.644 total time=  14.7s
[CV 3/3] END ......C=1, gamma=scale, kernel=rbf;, score=0.573 total time=  15.4s
[CV 3/3] END ......C=1, gamma=auto, kernel=poly;, score=0.678 total time=  11.7s
[CV 1/3] END ..C=10, gamma=scale, kernel=linear;, score=0.689 total time=  37.8s
[CV 3/3] END .....C=10, gamma=auto, kernel=poly;, score=0.676 total time=  10.8s
[CV 2/3] END .C=100, gamma=scale, kernel=linear;, score=0.688 total time=  43.7s
[CV 1/3] END .....C=100, gamma=auto, kernel=rbf;, score=0.655 total time=  16.3s
[CV 2/3] END ....C=0.1, gamma=scale, kernel=rbf;, score=0.230 total time=  21.0s
[CV 1/3] END .....C=0.1, gamma=auto, kernel=rbf;, score=0.332 total time=  23.8s
[CV 3/3] END .....C=1, gamma=scale, kernel=poly;, score=0.614 total time=  12.9s
[CV 2/3] END ....C=1, gamma=

In [141]:
result

,question,label,question_ID,gaussian_pred,SVM_pred
0,In this reaction which compound acts as the r...,2,34,0,0
1,Consider a scenario where Compound C is oxidiz...,1,32,0,0
2,In the context of biological redox reactions ...,0,27,0,0
3,Which assumption is necessary for biologists' ...,3,36,1,0
4,Which of the following best describes the role...,2,35,0,0
...,...,...,...,...,...
75,A molecule has a functional group with a pKa o...,3,23,0,2
76,Which statement best predicts the behavior of ...,0,24,0,0
77,A scientist is studying a biomolecule with a c...,1,25,0,2
78,If the enzyme is placed in a solution with a p...,2,26,0,2


In [142]:
## Logistic Regression

In [143]:
lr = LogisticRegression()
lr_gs = GridSearchCV(lr, params_lr, scoring="f1_macro", n_jobs=-1, cv=3, verbose=3)
start_time = time.time()
lr_gs.fit(train_x, train_y)
end_time = time.time()

print(f"Grid Search took {end_time - start_time:.2f} seconds")

# Predict with the optimized model
pred_y_lr = lr_gs.predict(test_x)

print(f"Best parameters: {lr_gs.best_params_}")
print(f"Best score: {lr_gs.best_score_}")

accuracy = accuracy_score(test_y, pred_y_lr)
print(f"Accuracy on the test set: {accuracy:.2f}")




Fitting 3 folds for each of 108 candidates, totalling 324 fits


/opt/anaconda3/envs/Diss/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/envs/Diss/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/envs/Diss/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/envs/Diss/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/envs/Diss/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/envs/Diss/lib/python3.10/site-packages/s

Grid Search took 491.17 seconds
Best parameters: {'C': 10, 'max_iter': 500, 'penalty': 'l1', 'solver': 'liblinear', 'tol': 0.01}
Best score: 0.7965884681228217
Accuracy on the test set: 0.80


In [144]:
pred_y_lr = lr_gs.predict(combined_x_auto)
accuracy = accuracy_score(y_auto, pred_y_lr)
print(f"Accuracy on Generated test set: {accuracy:.2f} \n \n")
result['lr_pred']= pred_y_lr

Accuracy on Generated test set: 0.28 
 



In [145]:
## Random Forest

In [147]:
rf = RandomForestClassifier()
rf_gs = GridSearchCV(rf, params_rf, scoring="f1_macro", n_jobs=-1, cv=3, verbose=3)
rf_gs.fit(train_x, train_y)
# Predict with the optimized model
pred_y_rf = rf_gs.predict(test_x)

print(f"Best parameters: {rf_gs.best_params_}")
print(f"Best score: {rf_gs.best_score_}")

accuracy = accuracy_score(test_y, pred_y_rf)
print(f"Accuracy on the test set: {accuracy:.2f}")


Fitting 3 folds for each of 324 candidates, totalling 972 fits
[CV 3/3] END C=0.1, max_iter=200, penalty=l1, solver=saga, tol=0.01;, score=0.630 total time=  18.0s
[CV 1/3] END C=0.1, max_iter=200, penalty=l1, solver=liblinear, tol=0.01;, score=0.660 total time=   1.6s
[CV 3/3] END C=0.1, max_iter=200, penalty=l1, solver=liblinear, tol=0.01;, score=0.643 total time=   1.2s
[CV 2/3] END C=0.1, max_iter=200, penalty=l1, solver=liblinear, tol=0.001;, score=0.627 total time=   1.3s
[CV 1/3] END C=0.1, max_iter=200, penalty=l1, solver=liblinear, tol=0.0001;, score=0.665 total time=   2.1s
[CV 3/3] END C=0.1, max_iter=200, penalty=l1, solver=liblinear, tol=0.0001;, score=0.651 total time=   2.0s
[CV 2/3] END C=0.1, max_iter=200, penalty=l2, solver=saga, tol=0.01;, score=0.618 total time=  13.5s
[CV 1/3] END C=0.1, max_iter=200, penalty=l2, solver=saga, tol=0.001;, score=0.662 total time=  46.5s
[CV 3/3] END C=0.1, max_iter=500, penalty=l1, solver=saga, tol=0.01;, score=0.630 total time=  17.

/opt/anaconda3/envs/Diss/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
486 fits failed out of a total of 972.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
386 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/anaconda3/envs/Diss/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/anaconda3/envs/Diss/lib/python3.10/site-packages/sklearn/base.py", line 1466, in wrapper
    estimator._validate_params()
  File "/opt/anaconda3/envs/Diss/lib/python3.10/site-packages/sklearn/base.py", line 666, in _validate_params
    validate_parameter_constraints(
  Fi

Best parameters: {'bootstrap': False, 'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 250}
Best score: 0.7885414148824452
Accuracy on the test set: 0.81


In [148]:
pred_y_rf = rf_gs.predict(combined_x_auto)
accuracy = accuracy_score(y_auto, pred_y_rf)
print(f"Accuracy on Generated test set: {accuracy:.2f} \n \n")
result['rf_pred']= pred_y_rf

Accuracy on Generated test set: 0.26 
 



In [149]:
## XGBoost

xgb = XGBClassifier()
xgb_gs = GridSearchCV(xgb, params_xgb, scoring="f1_macro", n_jobs=-1, cv=2, verbose=3)
xgb_gs.fit(train_x, train_y)
# Predict with the optimized model
pred_y_xgb = xgb_gs.predict(test_x)

print(f"Best parameters: {xgb_gs.best_params_}")
print(f"Best score: {xgb_gs.best_score_}")

accuracy = accuracy_score(test_y, pred_y_xgb)
print(f"Accuracy on the test set: {accuracy:.2f}")



Fitting 2 folds for each of 24 candidates, totalling 48 fits
Best parameters: {'gamma': 0.5, 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 100}
Best score: 0.7531204985362708
Accuracy on the test set: 0.83


In [150]:
pred_y_xgb = xgb_gs.predict(combined_x_auto)
accuracy = accuracy_score(y_auto, pred_y_xgb)
print(f"Accuracy on Generated set: {accuracy:.2f} \n \n")
result['xgb_pred']= pred_y_xgb

Accuracy on Generated set: 0.31 
 



In [151]:
performancePrinter(xgb_gs.predict(combined_x_auto), y_auto)

Accuracy Score ->  0.3125
Kappa Score ->  0.08333333333333337
ROC AUC Score -> Cannot calculate (probability scores required)
F1 Score ->  0.26778074866310164
Classification report -> 
               precision    recall  f1-score   support

           0       0.45      0.36      0.40        25
           1       0.35      0.29      0.32        24
           2       0.45      0.29      0.35        31
           3       0.00      0.00      0.00         0

    accuracy                           0.31        80
   macro avg       0.31      0.24      0.27        80
weighted avg       0.42      0.31      0.36        80



/opt/anaconda3/envs/Diss/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/envs/Diss/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/envs/Diss/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [152]:
np.unique(train_y)

array([0, 1, 2, 3])

In [175]:
result.to_excel('full_predictions.xlsx')

[CV 3/3] END bootstrap=False, max_depth=None, max_features=auto, min_samples_leaf=4, min_samples_split=10, n_estimators=250;, score=nan total time=   0.0s
[CV 3/3] END bootstrap=False, max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=50;, score=0.772 total time=   0.9s
[CV 2/3] END bootstrap=False, max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=50;, score=0.786 total time=   1.1s
[CV 3/3] END bootstrap=False, max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=100;, score=0.773 total time=   1.9s
[CV 2/3] END bootstrap=False, max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimators=50;, score=0.783 total time=   1.0s
[CV 1/3] END bootstrap=False, max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimators=250;, score=0.785 total time=   3.9s
[CV 1/3] END bootstrap=False, max_depth=None, max_features=sqrt, m

In [154]:
result

,question,label,question_ID,gaussian_pred,SVM_pred,lr_pred,rf_pred,xgb_pred
0,In this reaction which compound acts as the r...,2,34,0,0,1,0,0
1,Consider a scenario where Compound C is oxidiz...,1,32,0,0,2,0,1
2,In the context of biological redox reactions ...,0,27,0,0,1,0,2
3,Which assumption is necessary for biologists' ...,3,36,1,0,2,0,1
4,Which of the following best describes the role...,2,35,0,0,1,0,0
...,...,...,...,...,...,...,...,...
75,A molecule has a functional group with a pKa o...,3,23,0,2,1,0,1
76,Which statement best predicts the behavior of ...,0,24,0,0,1,0,1
77,A scientist is studying a biomolecule with a c...,1,25,0,2,1,0,1
78,If the enzyme is placed in a solution with a p...,2,26,0,2,2,0,2


In [155]:
#Random Forest
performancePrinter(rf_gs.predict(combined_x_auto), y_auto)

Accuracy Score ->  0.2625
Kappa Score ->  0.01666666666666672
ROC AUC Score -> Cannot calculate (probability scores required)
F1 Score ->  0.12690230731467844
Classification report -> 
               precision    recall  f1-score   support

           0       1.00      0.26      0.41        77
           1       0.00      0.00      0.00         2
           2       0.05      1.00      0.10         1
           3       0.00      0.00      0.00         0

    accuracy                           0.26        80
   macro avg       0.26      0.31      0.13        80
weighted avg       0.96      0.26      0.40        80



/opt/anaconda3/envs/Diss/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/envs/Diss/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/envs/Diss/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [156]:
#log regression
performancePrinter(lr_gs.predict(combined_x_auto), y_auto)

Accuracy Score ->  0.275
Kappa Score ->  0.033333333333333326
ROC AUC Score -> Cannot calculate (probability scores required)
F1 Score ->  0.21109164669002398
Classification report -> 
               precision    recall  f1-score   support

           0       0.05      0.11      0.07         9
           1       0.70      0.25      0.36        57
           2       0.35      0.50      0.41        14
           3       0.00      0.00      0.00         0

    accuracy                           0.28        80
   macro avg       0.28      0.21      0.21        80
weighted avg       0.57      0.28      0.34        80



/opt/anaconda3/envs/Diss/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/envs/Diss/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/envs/Diss/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


ValueError: X has 3119 features, but GaussianNB is expecting 1500 features as input.